# Clustering and differential expression analysis using scVI-tools

In [ ]:
import matplotlib.pyplot as plt
import scanpy as sc
import pandas as pd
import numpy as np
import json
import anndata
import scvi
import os

In [ ]:
dataset_name = 'PBMC1' # modify this
data_in_path = './data/{}/filtered/10X/'.format(dataset_name)
data_out_path = './data/{}/scvi-tools/'.format(dataset_name)
default_path = '{}default/'.format(data_out_path)
celltypist_path = '{}celltypist/'.format(data_out_path)
antibody_path = '{}antibody/'.format(data_out_path)
nclusters_celltypist_path = './data/{}/celltypist/nclusters.json'.format(dataset_name)
nclusters_antibody_path = './data/{}/antibody_annotation/nclusters_postproc.json'.format(dataset_name)

top_number_of_markers = 500
min_cluster_size = 0

with open(nclusters_celltypist_path) as f:
    nclusters_celltypist = json.load(f)['nclusters']
with open(nclusters_antibody_path) as f:
    nclusters_antibody = json.load(f)['nclusters']

nclusters_threshold = 1/10
min_ncluster_celltypist = round(nclusters_celltypist - nclusters_threshold*nclusters_celltypist)
max_ncluster_celltypist = round(nclusters_celltypist + nclusters_threshold*nclusters_celltypist)
min_ncluster_antibody = round(nclusters_antibody - nclusters_threshold*nclusters_antibody)
max_ncluster_antibody = round(nclusters_antibody + nclusters_threshold*nclusters_antibody)

if not os.path.exists(default_path):
    os.makedirs(default_path)

if not os.path.exists(celltypist_path):
    os.makedirs(celltypist_path)

if not os.path.exists(antibody_path):
    os.makedirs(antibody_path)

## Data loading and preparation

Dataset loading 

In [ ]:
adata = sc.read_10x_mtx(
    data_in_path,
    var_names='gene_symbols',
    cache=False
)
adata.var_names_make_unique()
adata

Studying feature variance

In [ ]:
matrix = adata.X
matrix = matrix.todense()
neg_variances = np.sort(-np.var(matrix, axis=0))
sorted_log_variances = [np.log(-i) for i in neg_variances.T][:2000]
plt.scatter([i for i in range(len(sorted_log_variances))], sorted_log_variances)

Data normalization

In [ ]:
adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata

Feature selection

In [ ]:
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=500, # chosen based on elbow plot above
    subset=True,
    layer="counts",
    flavor="seurat_v3",
)
sc.pl.highly_variable_genes(adata)

## Model training

In [ ]:
scvi.model.SCVI.setup_anndata(
    adata,
    layer="counts",
)

model = scvi.model.SCVI(adata)
model.train(use_gpu=False)

## Clustering

Save latent representation

In [ ]:
latent = model.get_latent_representation()
adata.obsm["X_scVI"] = latent
adata.layers["scvi_normalized"] = model.get_normalized_expression()

Plot pca explained variance ratio to choose number of components

In [ ]:
sc.tl.pca(adata, svd_solver="arpack", n_comps=20, use_highly_variable=True)
sc.pl.pca_variance_ratio(adata, log=True)

### Clustering with default parameters

Perform the clustering

In [ ]:
sc.pp.neighbors(
    adata,
    n_pcs=10, # chosen based on elbow plot above
    use_rep="X_scVI"
)
sc.tl.leiden(adata, key_added="leiden_scVI")

Visualize the clustering in the PCA space

In [ ]:
sc.pl.pca(
    adata,
    color=["leiden_scVI"],
    size=20,
)

Save the clustering results

In [ ]:
default_clusters_df = pd.DataFrame(adata.obs['leiden_scVI'])
default_clusters_df = default_clusters_df.rename(columns={'leiden_scVI': 'cluster'})
default_clusters_df.index.names = ['cell']
default_clusters_df['cluster'] = default_clusters_df['cluster'].astype(int) + 1
default_clusters_df.to_csv(default_path+'clustering_labels.csv')
default_num_clusters = len(default_clusters_df['cluster'].unique())

## Differential expression on default clusters

Perform the differential expression analysis

In [ ]:
test_cells = np.array([])
adata_train = adata.copy()
for cluster in range(default_num_clusters):
    cluster_cells = default_clusters_df[default_clusters_df['cluster'] == cluster]
    cluster_cells = cluster_cells.sample(frac=0.2)
    test_cells = np.concatenate([test_cells, cluster_cells.index.values])
    adata_train = adata_train[~adata_train.obs.index.isin(cluster_cells.index)].copy()
pd.DataFrame(test_cells, columns=['cell']).to_csv(f'{default_path}/test_cells.csv', index=False)
model.adata = adata_train
default_de_df = model.differential_expression(
    groupby="leiden_scVI",
)

Save markers

In [ ]:
default_markers = {}
default_cats = adata_train.obs.leiden_scVI.cat.categories
for i, c in enumerate(default_cats):
    cell_type_df = default_de_df.loc[default_de_df.group1 == c]
    # if uncommented there are not enough genes
    #cell_type_df = cell_type_df[cell_type_df["bayes_factor"] > 3]
    #cell_type_df = cell_type_df[cell_type_df["non_zeros_proportion1"] > 0.1]
    default_markers[c] = cell_type_df.sort_values('lfc_mean', ascending=False).index.tolist()[:top_number_of_markers]

default_markers_df = pd.DataFrame(columns=['gene', 'cluster', 'rank'])
for i in range(default_num_clusters):
    tmp_list = [ [default_markers[str(i)][j], i, j] for j in range(top_number_of_markers)]
    tmp_df = pd.DataFrame(tmp_list, columns=['gene', 'cluster', 'rank'])
    default_markers_df = pd.concat([default_markers_df, tmp_df])
default_markers_df['cluster'] += 1
default_markers_df['rank'] += 1
default_markers_df.to_csv(default_path+'markers.csv', index=False)

### Clustering tuning resolution according to celltypist

In [ ]:
print("Nummber of clusters to find: {}".format(nclusters_celltypist))

In [ ]:
# get ids of clusters bigger than 40 cells
mapping = pd.read_csv('./data/{}/celltypist/celltypist_mapping.csv'.format(dataset_name).format(dataset_name))
counts = pd.read_csv('./data/{}/celltypist/celltypist_annotation_counts.csv'.format(dataset_name))
mapping_counts = mapping.merge(counts, left_on='go', right_on='cluster.ids')
mapping_counts = mapping_counts[mapping_counts['count'] > min_cluster_size]
clusters_ids_to_keep = mapping_counts['id']

# get barcodes of cells in clusters bigger than 40 cells
celltypist_labels_df = pd.read_csv('./data/{}/celltypist/celltypist_labels.csv'.format(dataset_name))
celltypist_labels_df = celltypist_labels_df[celltypist_labels_df['cluster.ids'].isin(clusters_ids_to_keep)]
barcodes_to_keep = celltypist_labels_df['cell']
barcodes_to_keep = [barcode[:-2] for barcode in barcodes_to_keep]
subset_cells = adata.obs_names.isin(barcodes_to_keep)
adata_celltypist = adata[subset_cells, :].copy()

## Model training

In [ ]:
scvi.model.SCVI.setup_anndata(
    adata_celltypist,
    layer="counts",
)

model = scvi.model.SCVI(adata_celltypist)
model.train(use_gpu=False)

Save latent representation

In [ ]:
latent = model.get_latent_representation()
adata_celltypist.obsm["X_scVI"] = latent
adata_celltypist.layers["scvi_normalized"] = model.get_normalized_expression()

Plot pca explained variance ratio to choose number of components

In [ ]:
sc.tl.pca(adata_celltypist, svd_solver="arpack", n_comps=20, use_highly_variable=True)
sc.pl.pca_variance_ratio(adata_celltypist, log=True)

Perform the clustering

In [ ]:
sc.pp.neighbors(
    adata_celltypist,
    n_pcs=10, # chosen based on elbow plot above
    use_rep="X_scVI"
)
sc.tl.leiden(adata_celltypist, key_added="leiden_scVI")

max_resolution = 3
min_resolution = 0

while True:
    resolution = (max_resolution + min_resolution)/2
    print("Trying resolution: {}".format(resolution))
    sc.tl.leiden(adata_celltypist, key_added="leiden_scVI", resolution=resolution)
    num_clusters = adata_celltypist.obs.leiden_scVI.values.categories.nunique()
    print("Number of clusters found: {}".format(num_clusters))

    if num_clusters >= min_ncluster_celltypist and num_clusters <= max_ncluster_celltypist:
        break
    elif num_clusters <  min_ncluster_celltypist:
        min_resolution = resolution
    else:
        max_resolution = resolution

Visualize the clustering in the PCA space

In [ ]:
sc.pl.pca(
    adata_celltypist,
    color=["leiden_scVI"],
    size=20,
)

Save the clustering results

In [ ]:
celltypist_clusters_df = pd.DataFrame(adata_celltypist.obs['leiden_scVI'])
celltypist_clusters_df = celltypist_clusters_df.rename(columns={'leiden_scVI': 'cluster'})
celltypist_clusters_df.index.names = ['cell']
celltypist_clusters_df['cluster'] = celltypist_clusters_df['cluster'].astype(int) + 1
celltypist_clusters_df.to_csv(celltypist_path+'clustering_labels.csv')
celltypist_num_clusters = len(celltypist_clusters_df['cluster'].unique())

## Differential expression on clusters tuned according to celltypist

Perform the differential expression analysis

In [ ]:
celltypist_de_df = model.differential_expression(
    groupby="leiden_scVI",
)

Save markers

In [ ]:
celltypist_markers = {}
cats = adata_celltypist.obs.leiden_scVI.cat.categories
for i, c in enumerate(cats):
    cell_type_df = celltypist_de_df.loc[celltypist_de_df.group1 == c]
    # if uncommented there are not enough genes
    #cell_type_df = cell_type_df[cell_type_df["bayes_factor"] > 3]
    #cell_type_df = cell_type_df[cell_type_df["non_zeros_proportion1"] > 0.1]
    celltypist_markers[c] = cell_type_df.sort_values('lfc_mean', ascending=False).index.tolist()[:top_number_of_markers]

celltypist_markers_df = pd.DataFrame(columns=['gene', 'cluster', 'rank'])
for i in range(celltypist_num_clusters):
    tmp_list = [ [celltypist_markers[str(i)][j], i, j] for j in range(top_number_of_markers)]
    tmp_df = pd.DataFrame(tmp_list, columns=['gene', 'cluster', 'rank'])
    celltypist_markers_df = pd.concat([celltypist_markers_df, tmp_df],  ignore_index=True)

celltypist_markers_df['cluster'] += 1
celltypist_markers_df['rank'] += 1
celltypist_markers_df.to_csv(celltypist_path+'markers.csv', index=False)

### Clustering tuning resolution according to protein surface

In [ ]:
print("Nummber of clusters to find: {}".format(nclusters_antibody))

In [ ]:
# get barcodes of cells labelled using protein surface
antibody_labels_df = pd.read_csv('./data/{}/antibody_annotation/antibody_labels_postproc.csv'.format(dataset_name))
barcodes_to_keep = antibody_labels_df['cell']

subset_cells = adata.obs_names.isin(barcodes_to_keep)
adata_antibody = adata[subset_cells, :].copy()

## Model training

In [ ]:
scvi.model.SCVI.setup_anndata(
    adata_antibody,
    layer="counts",
)

model = scvi.model.SCVI(adata_antibody)
model.train(use_gpu=False)

Save latent representation

In [ ]:
latent = model.get_latent_representation()
adata_antibody.obsm["X_scVI"] = latent
adata_antibody.layers["scvi_normalized"] = model.get_normalized_expression()

Plot pca explained variance ratio to choose number of components

In [ ]:
sc.tl.pca(adata_antibody, svd_solver="arpack", n_comps=20, use_highly_variable=True)
sc.pl.pca_variance_ratio(adata_antibody, log=True)

Perform the clustering

In [ ]:
sc.pp.neighbors(
    adata_antibody,
    n_pcs=10, # chosen based on elbow plot above
    use_rep="X_scVI"
)
sc.tl.leiden(adata_antibody, key_added="leiden_scVI")

max_resolution = 3
min_resolution = 0

while True:
    resolution = (max_resolution + min_resolution)/2
    print("Trying resolution: {}".format(resolution))
    sc.tl.leiden(adata_antibody, key_added="leiden_scVI", resolution=resolution)
    num_clusters = adata_antibody.obs.leiden_scVI.values.categories.nunique()
    print("Number of clusters found: {}".format(num_clusters))

    if num_clusters >= min_ncluster_antibody and num_clusters <= max_ncluster_antibody:
        break
    elif num_clusters < min_ncluster_antibody:
        min_resolution = resolution
    else:
        max_resolution = resolution

Visualize the clustering in the PCA space

In [ ]:
sc.pl.pca(
    adata_antibody,
    color=["leiden_scVI"],
    size=20,
)

Save the clustering results

In [ ]:
antibody_clusters_df = pd.DataFrame(adata_antibody.obs['leiden_scVI'])
antibody_clusters_df = antibody_clusters_df.rename(columns={'leiden_scVI': 'cluster'})
antibody_clusters_df.index.names = ['cell']
antibody_clusters_df['cluster'] = antibody_clusters_df['cluster'].astype(int) + 1
antibody_clusters_df.to_csv(antibody_path+'clustering_labels.csv')
antibody_num_clusters = len(antibody_clusters_df['cluster'].unique())

## Differential expression on clusters tuned according to protein surface

Perform the differential expression analysis

In [ ]:
antibody_de_df = model.differential_expression(
    groupby="leiden_scVI",
)

Save markers

In [ ]:
antibody_markers = {}
cats = adata_antibody.obs.leiden_scVI.cat.categories
for i, c in enumerate(cats):
    cell_type_df = antibody_de_df.loc[antibody_de_df.group1 == c]
    # if uncommented there are not enough genes
    #cell_type_df = cell_type_df[cell_type_df["bayes_factor"] > 3]
    #cell_type_df = cell_type_df[cell_type_df["non_zeros_proportion1"] > 0.1]
    antibody_markers[c] = cell_type_df.sort_values('lfc_mean', ascending=False).index.tolist()[:top_number_of_markers]

antibody_markers_df = pd.DataFrame(columns=['gene', 'cluster', 'rank'])
for i in range(antibody_num_clusters):
    tmp_list = [ [antibody_markers[str(i)][j], i, j] for j in range(top_number_of_markers)]
    tmp_df = pd.DataFrame(tmp_list, columns=['gene', 'cluster', 'rank'])
    antibody_markers_df = pd.concat([antibody_markers_df, tmp_df],  ignore_index=True)

antibody_markers_df['cluster'] += 1
antibody_markers_df['rank'] += 1
antibody_markers_df.to_csv(antibody_path+'markers.csv', index=False)